这个例子模拟Google的Sycamore量子计算机，30量子比特14层量子线路，我们可以给出保真度接近1的振幅。

In [1]:
from tensorq import QuantumCircuit
n = 30
qc_n = QuantumCircuit(n = n, fname = 'circuit_n30_m14_s0_e0_pEFGH')

n = 30表示30量子比特。circuit_n30_m14_s0_e0_pEFGH是线路文件，可以在[Google's data repository for the Sycamore circuits.](https://datadryad.org/stash/dataset/doi:10.5061/dryad.k6t1rj8)下载

In [2]:
from tensorq import search_order
contract_scheme = search_order(n = n, device = 'cuda', qc = qc_n, sc_target = 24, bitstrings_txt = 'amplitudes_n30_m14_s0_e0_pEFGH_10000.txt', max_bitstrings=1000)

Finish find contraction tree, next construct the scheme.
Finish construct the scheme.
time complexity (tc,log10), space complexity (sc,log2), memory complexity (mc) =  (12.014205108939997, 30.0, 10.304655927314002)


其中，n = n表示n个比特；device = 'cuda' 表示用GPU进行计算，用CPU计算使用device = 'cpu'；qc = qc_n 输入之前定义的量子线路。

sc_target = 24表示限制最大张量小于24GB内存，这是tensorq中很核心的一个参数，可以根据自己的设备内存进行设置。同样的缩并任务，内存越大时间复杂度越低，内存越小将会对张量做更多的分解导致更多的浮点运算，这是用时间换空间的办法解决量子模拟中的指数墙困难。

bitstrings_txt = 'amplitudes_n30_m14_s0_e0_pEFGH_10000.txt' 表示将我们需要计算振幅的比特串写在`amplitudes_n30_m14_s0_e0_pEFGH_10000.txt`文件里面。

max_bitstrings=10000，表示计算10000个比特串的振幅。

search_order会打印该缩并方案的复杂度，时间复杂度为浮点数运算次数，以10为底的对数表示；空间复杂度为最大张量的元素个数（与sc_target单位并不相同），以2为底的对数表示；空间复杂度为最大张量元素个数乘以单个数据的存储大小（与数据类型有关）。


用TensorQ计算振幅

In [3]:
from tensorq import contraction_single_task
tensors, scheme, slicing_indices, bitstrings = contract_scheme
result = contraction_single_task(
    tensors,
    scheme,
    slicing_indices,
    # task_id=0,
    # n_sub_task=2**12
)
print('fianl amplitue:', result)

fianl amplitue: tensor([ 4.3482e-08-1.7836e-05j, -3.2258e-05+8.9927e-05j,
         1.0095e-05+1.8853e-05j, -2.1370e-05-4.6581e-06j,
        -2.2554e-06-6.5780e-06j,  2.2370e-05+1.8043e-05j,
        -6.0554e-07-4.0699e-06j, -1.7893e-05+2.4557e-05j,
         5.6588e-06+2.3819e-05j,  1.9241e-05+3.9711e-05j,
        -5.4009e-06-4.0499e-06j, -7.0265e-07+9.9225e-06j,
        -1.3044e-05+1.0328e-07j,  1.7230e-05+9.6520e-06j,
         2.1250e-05+4.7519e-05j, -2.5736e-05+8.1803e-06j,
        -6.2696e-06-6.8121e-06j, -8.9922e-06+2.4323e-05j,
        -1.7820e-05-4.5324e-05j,  3.7944e-05-2.9249e-05j,
        -8.5800e-07+7.0216e-06j, -2.3244e-05+9.1181e-06j,
        -7.9062e-06+2.1183e-05j,  1.2550e-05-1.0547e-05j,
        -1.4457e-05-2.7044e-05j,  1.2294e-05+1.0695e-05j,
         1.6134e-05+5.4860e-06j,  6.3205e-06-1.6330e-05j,
         1.5391e-05+2.1379e-05j,  1.7455e-05-1.3334e-05j,
         4.0696e-05+7.6519e-06j, -6.2703e-07+1.7520e-05j,
         1.0775e-05-2.1403e-05j,  1.8035e-05-2.8506e-05j

contract_scheme中包含了缩并张量tensors，缩并步骤scheme，切片指标slicing_indices（与张量分解有关），计算振幅的比特串bitstrings，将这些变量输入到contraction_single_task，就会返回张量网络的缩并结果，就是全部末态比特串的振幅。


从'amplitudes_n30_m14_s0_e0_pEFGH_10000.txt'文件中读取相应的精确振幅以及比特串

In [4]:
from tensorq import read_samples
import numpy as np
data = read_samples('amplitudes_n30_m14_s0_e0_pEFGH_10000.txt')
max_bitstrings = 10000
bitstrings_exact = [data[i][0] for i in range(max_bitstrings)]
amplitude_exact = np.array([data[i][1] for i in range(max_bitstrings)])

将精确振幅的顺序按照我们计算的比特串顺序排列

In [5]:
import torch
amplitude_exact_1000 = []
for i in range(1000):
    amplitude_exact_1000.append(amplitude_exact[bitstrings_exact.index(bitstrings[i])])
amplitude_exact_1000 = torch.tensor(amplitude_exact_1000, dtype=torch.complex64)

计算TensorQ结果的保真度。

In [6]:
fidelity = (
        (amplitude_exact_1000.conj() @ result.reshape(-1)).abs() /
        (amplitude_exact_1000.abs().square().sum().sqrt() * result.abs().square().sum().sqrt())
    ).square().item()
fidelity

0.9999997615814209

TensorQ结果的保真度接近1。

计算一组均匀分布的白噪音的保真度

In [7]:
noisy = torch.tensor([1/np.sqrt(len(result)) for i in range(len(result))], dtype=torch.complex64)
fidelity2 = (
        (amplitude_exact_1000.conj() @ noisy.reshape(-1)).abs() /
        (amplitude_exact_1000.abs().square().sum().sqrt() * noisy.abs().square().sum().sqrt())
    ).square().item()
fidelity2

0.0013713767984881997

白噪音的保真度远小于1